In [27]:
# %% [markdown]
# # Phân tích KPI Mạng Di động (15 Mins)
#
# Notebook này phân tích dữ liệu KPI được trích xuất từ file `kpi_15_mins.csv`.
# Mục tiêu là làm sạch dữ liệu, trực quan hóa các xu hướng chính và xác định các cell/enodeb hoạt động hàng đầu.

# %%
# --- Cell 1: Tải các thư viện cần thiết ---
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np # Cần cho việc xử lý NaN

# Cấu hình để biểu đồ hiển thị ngay trong notebook
%matplotlib inline
# Cấu hình hiển thị cho Pandas
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

print("Các thư viện đã được tải thành công.")

# %%

Các thư viện đã được tải thành công.


In [ ]:
# --- Cell 2 (Sửa lỗi): Tải dữ liệu với header=0 ---

# Tên các cột bạn đã cung cấp (Dùng để tham khảo)
# column_names = [
#     'update_time', 'enodeb', 'cell_name', 'ps_traffic_mb',
#     'avg_rrc_connected_user', 'prb_dl_used', 'prb_dl_available_total', 'date_hour'
# ]

# Tên file
file_name = 'kpi_15_mins.csv'

try:
    # *** SỬA LỖI QUAN TRỌNG ***
    # header=0 bảo Pandas đọc hàng đầu tiên (index 0) làm tiêu đề.
    # Chúng ta không cần 'names=column_names' nữa vì tên đã có trong file.
    df = pd.read_csv(file_name, header=0)

    print(f"Đã tải file '{file_name}' thành công (sử dụng hàng 0 làm tiêu đề).")
    print("5 dòng dữ liệu đầu tiên:")
    print(df.head())
    print("\nKiểm tra kiểu dữ liệu (dtypes) sau khi tải:")
    print(df.info()) # Kiểm tra xem các cột số có phải là float/int không

except FileNotFoundError:
    print(f"LỖI: Không tìm thấy file '{file_name}'.")
except Exception as e:
    print(f"Đã xảy ra lỗi khi đọc file: {e}")

Đã tải file 'D://vhproj//power-saving//data//kpi_15_mins.csv' thành công.
5 dòng dữ liệu đầu tiên:
   update_time   enodeb        cell_name  ps_traffic_mb  avg_rrc_connected_user  prb_dl_used  prb_dl_available_total      date_hour
0  update_time   enodeb        cell_name  ps_traffic_mb  avg_rrc_connected_user  prb_dl_used  prb_dl_available_total      date_hour
1      00:00.0  EnodebA         EnodebA3         161.14                8.527778         67.5                      15  2025-10-11-00
2      00:00.0  EnodebA  EnodebA1111B031          733.1               22.038889        377.5                      15  2025-10-11-00
3      00:00.0  EnodebA    EnodebA61B281          141.4                7.872223        127.5                      15  2025-10-11-00
4      00:00.0  EnodebE    EnodebE61B282              0                       0            0                       0  2025-10-11-00


C:\Users\firek\AppData\Local\Temp\ipykernel_17020\1680432396.py:16: DtypeWarning: Columns (3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_name, header=None, names=column_names)


In [29]:
# --- Cell 3: Kiểm tra dữ liệu ban đầu ---
if 'df' in locals():
    print("\n--- Thông tin DataFrame (df.info()) ---")
    # df.info() cho thấy kiểu dữ liệu và các giá trị không rỗng (non-null)
    df.info()

    print("\n--- Thống kê mô tả (df.describe()) ---")
    # df.describe() cung cấp thống kê tóm tắt cho các cột số
    print(df.describe())

# %%


--- Thông tin DataFrame (df.info()) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 313187 entries, 0 to 313186
Data columns (total 8 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   update_time             313187 non-null  object
 1   enodeb                  313187 non-null  object
 2   cell_name               313187 non-null  object
 3   ps_traffic_mb           313169 non-null  object
 4   avg_rrc_connected_user  313187 non-null  object
 5   prb_dl_used             312998 non-null  object
 6   prb_dl_available_total  312998 non-null  object
 7   date_hour               313187 non-null  object
dtypes: object(8)
memory usage: 19.1+ MB

--- Thống kê mô tả (df.describe()) ---
       update_time   enodeb   cell_name  ps_traffic_mb  avg_rrc_connected_user  prb_dl_used  prb_dl_available_total      date_hour
count       313187   313187      313187       313169.0                313187.0     312998.0                

In [30]:
# --- Cell 4 (Sửa lỗi - Thêm Debug): Làm sạch dữ liệu và Kỹ thuật đặc trưng ---
if 'df' in locals():
    print("Bắt đầu làm sạch dữ liệu và kỹ thuật đặc trưng...")

    try:
        # 1. Kết hợp `date_hour` và `update_time` thành một cột datetime đầy đủ
        df['minute_str'] = df['update_time'].str.split(':').str[0].str.strip()
        df['full_datetime_str'] = df['date_hour'].str.strip() + '-' + df['minute_str']

        # *** (DÒNG GỠ LỖI MỚI) ***
        # Hãy in ra 5 giá trị đầu tiên để xem định dạng thực tế
        print("\n--- 5 giá trị 'full_datetime_str' đầu tiên để kiểm tra format: ---")
        print(df['full_datetime_str'].head(5))
        # *** (KẾT THÚC DÒNG GỠ LỖI) ***

        # 2. Chuyển đổi chuỗi kết hợp thành đối tượng datetime
        # *** KIỂM TRA LẠI ĐỊNH DẠNG NÀY ***
        # Nó phải khớp 100% với output ở trên
        my_format = '%Y-%m-%d-%H-%M' 
        df['timestamp'] = pd.to_datetime(df['full_datetime_str'], format=my_format, errors='coerce')

        # 3. Xử lý các dòng bị lỗi (NaT)
        if df['timestamp'].isnull().any():
            invalid_rows = df['timestamp'].isnull().sum()
            print(f"PHÁT HIỆN: {invalid_rows} dòng có định dạng thời gian không hợp lệ. Đang loại bỏ...")
            df = df.dropna(subset=['timestamp'])

        if df.empty:
            print(f"LỖI: Toàn bộ dữ liệu đã bị loại bỏ.")
            print(f"Định dạng bạn dùng ('{my_format}') có thể KHÔNG KHỚP với dữ liệu.")
            print("Hãy so sánh nó với 5 giá trị đã in ở trên.")
        else:
            # 4. Đặt 'timestamp' làm chỉ mục (index) của DataFrame
            df = df.set_index('timestamp')

            # 5. Tạo KPI mới 'prb_utilization'
            print("Tạo KPI mới 'prb_utilization'...")
            df['prb_available_clean'] = df['prb_dl_available_total'].replace(0, np.nan)
            df['prb_utilization'] = (df['prb_dl_used'] / df['prb_available_clean']) * 100
            df['prb_utilization'] = df['prb_utilization'].fillna(0)
            
            # 6. Bỏ các cột tạm thời
            columns_to_drop = ['update_time', 'date_hour', 'minute_str', 'full_datetime_str', 'prb_available_clean']
            df = df.drop(columns=columns_to_drop)

            print("\nHoàn tất làm sạch dữ liệu.")
            print("\nThông tin DataFrame sau khi làm sạch:")
            df.info()

    except Exception as e:
        print(f"Đã xảy ra lỗi nghiêm trọng trong quá trình làm sạch: {e}")
else:
    print("Lỗi: DataFrame 'df' không tồn tại. Vui lòng chạy lại Cell 2 để tải dữ liệu.")

Bắt đầu làm sạch dữ liệu và kỹ thuật đặc trưng...

--- 5 giá trị 'full_datetime_str' đầu tiên để kiểm tra format: ---
0    date_hour-update_time
1         2025-10-11-00-00
2         2025-10-11-00-00
3         2025-10-11-00-00
4         2025-10-11-00-00
Name: full_datetime_str, dtype: object
PHÁT HIỆN: 1 dòng có định dạng thời gian không hợp lệ. Đang loại bỏ...
Tạo KPI mới 'prb_utilization'...
Đã xảy ra lỗi nghiêm trọng trong quá trình làm sạch: unsupported operand type(s) for /: 'str' and 'str'


In [32]:


# %%
# --- Cell 5: Phân tích chuỗi thời gian (Tổng quan) ---
# Chúng ta sẽ tổng hợp dữ liệu theo giờ để xem xu hướng tổng thể
if 'df' in locals():
    print("Tạo biểu đồ chuỗi thời gian theo giờ...")

    # 'H' = resample by Hour (lấy mẫu lại theo giờ)
    # .sum() cho traffic, .mean() cho users và utilization
    df_hourly = df.resample('H').agg({
        'ps_traffic_mb': 'sum',
        'avg_rrc_connected_user': 'mean',
        'prb_utilization': 'mean'
    })

    # Bỏ các giờ không có dữ liệu (nếu có)
    df_hourly = df_hourly[df_hourly['ps_traffic_mb'] > 0]

    # Vẽ 3 biểu đồ
    fig, axes = plt.subplots(3, 1, figsize=(18, 15), sharex=True)
    fig.suptitle('Tổng quan KPI theo giờ', fontsize=16)

    # Biểu đồ 1: PS Traffic
    axes[0].plot(df_hourly.index, df_hourly['ps_traffic_mb'], label='Tổng Traffic (MB)', color='blue')
    axes[0].set_ylabel('Tổng PS Traffic (MB)')
    axes[0].legend()
    axes[0].grid(True)

    # Biểu đồ 2: Average RRC Connected Users
    axes[1].plot(df_hourly.index, df_hourly['avg_rrc_connected_user'], label='TB Users', color='orange')
    axes[1].set_ylabel('TB RRC Connected Users')
    axes[1].legend()
    axes[1].grid(True)

    # Biểu đồ 3: Average PRB Utilization
    axes[2].plot(df_hourly.index, df_hourly['prb_utilization'], label='TB PRB Utilization (%)', color='green')
    axes[2].set_ylabel('TB PRB Utilization (%)')
    axes[2].set_xlabel('Thời gian')
    axes[2].legend()
    axes[2].grid(True)

    plt.tight_layout(rect=[0, 0.03, 1, 0.97])
    # plt.savefig('kpi_time_series.png') # Lưu biểu đồ ra file
    plt.show()

# %%

Tạo biểu đồ chuỗi thời gian theo giờ...


C:\Users\firek\AppData\Local\Temp\ipykernel_17020\2956925155.py:9: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_hourly = df.resample('H').agg({


KeyError: "Column(s) ['prb_utilization'] do not exist"

In [ ]:

# --- Cell 6: Phân tích theo hạng mục (EnodeB và Cell) ---
if 'df' in locals():
    print("Phân tích Top EnodeB và Cell...")

    # 1. Top EnodeBs theo tổng lưu lượng
    top_n = 15
    df_enodeb_traffic = df.groupby('enodeb')['ps_traffic_mb'].sum().sort_values(ascending=False)
    
    print(f"\n--- Top {top_n} EnodeB theo Tổng Traffic ---")
    print(df_enodeb_traffic.head(top_n))

    # Vẽ biểu đồ bar cho Top EnodeB
    plt.figure(figsize=(15, 7))
    df_enodeb_traffic.head(top_n).plot(kind='bar', color='skyblue')
    plt.title(f'Top {top_n} EnodeB theo Tổng Traffic (MB)')
    plt.ylabel('Tổng Traffic (MB)')
    plt.xlabel('EnodeB')
    plt.xticks(rotation=45, ha='right')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    # plt.savefig('top_enodeb_traffic.png')
    plt.show()

    # 2. Top Cells theo tổng lưu lượng
    top_n_cells = 20
    df_cell_traffic = df.groupby('cell_name')['ps_traffic_mb'].sum().sort_values(ascending=False)

    print(f"\n--- Top {top_n_cells} Cell theo Tổng Traffic ---")
    print(df_cell_traffic.head(top_n_cells))
    
    # Vẽ biểu đồ bar cho Top Cell
    plt.figure(figsize=(15, 7))
    df_cell_traffic.head(top_n_cells).plot(kind='bar', color='lightgreen')
    plt.title(f'Top {top_n_cells} Cell theo Tổng Traffic (MB)')
    plt.ylabel('Tổng Traffic (MB)')
    plt.xlabel('Cell Name')
    plt.xticks(rotation=90)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    # plt.savefig('top_cell_traffic.png')
    plt.show()

# %%

In [ ]:

# --- Cell 7: Phân tích mối tương quan (Correlation) ---
if 'df' in locals():
    print("Phân tích mối tương quan giữa các KPI...")
    
    # Chọn các cột số để tính tương quan
    numeric_cols = ['ps_traffic_mb', 'avg_rrc_connected_user', 'prb_dl_used', 'prb_utilization', 'prb_dl_available_total']
    
    # Tính ma trận tương quan
    correlation_matrix = df[numeric_cols].corr()
    
    print("\n--- Ma trận tương quan ---")
    print(correlation_matrix)
    
    # Vẽ Heatmap (Bản đồ nhiệt)
    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
    plt.title('Heatmap tương quan giữa các KPI')
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    # plt.savefig('kpi_correlation_heatmap.png')
    plt.show()
    
    # Giải thích nhanh:
    # - Giá trị gần +1.0: Tương quan thuận mạnh (ví dụ: Traffic tăng thì Users cũng tăng)
    # - Giá trị gần -1.0: Tương quan nghịch mạnh
    # - Giá trị gần 0: Không có tương quan tuyến tính

# %%

In [ ]:

# --- Cell 8: Phân tích theo giờ trong ngày (Hour of Day) ---
# Giúp xác định giờ cao điểm (peak hours)
if 'df_hourly' in locals():
    print("Phân tích xu hướng trung bình theo giờ trong ngày...")

    # Thêm cột 'hour_of_day' vào df_hourly
    df_hourly['hour_of_day'] = df_hourly.index.hour
    
    # Tính trung bình các KPI cho mỗi giờ trong ngày (0-23)
    df_hod_avg = df_hourly.groupby('hour_of_day').mean()
    
    print("\n--- KPI trung bình theo giờ trong ngày ---")
    print(df_hod_avg[['ps_traffic_mb', 'avg_rrc_connected_user', 'prb_utilization']])

    # Vẽ biểu đồ đường
    fig, axes = plt.subplots(3, 1, figsize=(15, 12), sharex=True)
    fig.suptitle('Xu hướng KPI trung bình theo Giờ trong ngày', fontsize=16)
    
    # Các giờ trong ngày
    hours = df_hod_avg.index
    
    # Biểu đồ 1: Traffic
    axes[0].plot(hours, df_hod_avg['ps_traffic_mb'], label='TB Traffic (MB)', color='blue', marker='o')
    axes[0].set_ylabel('TB PS Traffic (MB)')
    axes[0].legend()
    axes[0].grid(True)

    # Biểu đồ 2: Users
    axes[1].plot(hours, df_hod_avg['avg_rrc_connected_user'], label='TB Users', color='orange', marker='o')
    axes[1].set_ylabel('TB RRC Connected Users')
    axes[1].legend()
    axes[1].grid(True)

    # Biểu đồ 3: PRB Utilization
    axes[2].plot(hours, df_hod_avg['prb_utilization'], label='TB PRB Utilization (%)', color='green', marker='o')
    axes[2].set_ylabel('TB PRB Utilization (%)')
    axes[2].set_xlabel('Giờ trong ngày (0-23)')
    axes[2].legend()
    axes[2].grid(True)
    
    # Đảm bảo trục x hiển thị tất cả các giờ
    plt.xticks(hours)
    
    plt.tight_layout(rect=[0, 0.03, 1, 0.97])
    # plt.savefig('kpi_hourly_avg_trend.png')
    plt.show()

# %% [markdown]
# # Kết thúc Phân tích
#
# Notebook này đã cung cấp các bước cơ bản để:
# 1. Tải và làm sạch dữ liệu.
# 2. Tạo cột `timestamp` và chỉ số `prb_utilization`.
# 3. Trực quan hóa xu hướng theo thời gian.
# 4. Xác định các EnodeB/Cell hàng đầu.
# 5. Phân tích tương quan và xu hướng theo giờ trong ngày.
#
# Từ đây, bạn có thể đi sâu hơn vào việc phân tích các cell cụ thể, phát hiện bất thường, hoặc dự đoán traffic.